In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize

# input_horizontal =range(32,64,32)
# input_vertical =range(32,64,32)
# for input_x,input_y in product(input_horizontal,input_vertical):
shape_range =range(32,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    inputs = Input(shape=(shape_x, shape_y, 1))

    conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=2)(conv2)

    conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=2)(conv4)

    conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
    conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
    pool3 = MaxPooling2D(pool_size=2)(conv7)

    conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
    conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
    conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
    pool4 = MaxPooling2D(pool_size=2)(conv10)

    conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
    conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
    conv13 = Conv2D(256, (3,3), padding='same', activation='relu')(conv12)
    pool5 = MaxPooling2D(pool_size=2)(conv13)

    flat = Flatten()(pool5)

    fc1 = Dense(4096, activation='relu')(flat)
    fc2 = Dense(4096, activation='relu')(fc1)

    outputs = Dense(10, activation='softmax')(fc2)


    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])
    #history = model.fit(x_train, y_train, epochs=1)
    #model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
    #quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
            --model ./quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel_vgg16 \
            --net_name deploy
    os.rename("./xmodel_vgg16/deploy.xmodel","./xmodel_vgg16/vgg16_{input_x}_{input_y}.xmodel".format(input_x=shape_x,input_y=shape_y))

2022-06-10 16:33:04.311145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-06-10 16:33:04.311164: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-10 16:33:05.724145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-06-10 16:33:05.724174: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0     

NameError: name 'quantized_model' is not defined

In [ ]:
for input_x,input_y in product(input_horizontal,input_vertical):
    x_train = tf.image.resize(x_train, [input_x,input_y])
    x_test = tf.image.resize(x_test, [input_x,input_y])
    print(x_train.shape)
    print(x_train.dtype)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from keras_flops import get_flops

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('Training data: {}, {}'.format(x_train.shape, y_train.shape))
print('Test data: {}, {}'.format(x_test.shape, y_test.shape))
x_train = x_train[0:60]
y_train = y_train[0:60]
x_test = x_test[0:10]
y_test = y_test[0:10]

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

input_range =range(32,64,32)

for input_x in input_range:
    input_y = input_x
    x_train = tf.image.resize(x_train, [input_x,input_y])
    x_test = tf.image.resize(x_test, [input_x,input_y])


    inputs = Input(shape=(input_x, input_y, 1))

    # 卷积层和最大池化层
    conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
    conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=2)(conv2)

    conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=2)(conv4)

    conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
    conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
    pool3 = MaxPooling2D(pool_size=2)(conv7)

    conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
    conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
    conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
    pool4 = MaxPooling2D(pool_size=2)(conv10)

    conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
    conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
    conv13 = Conv2D(256, (3,3), padding='same', activation='relu')(conv12)
    pool5 = MaxPooling2D(pool_size=2)(conv13)

    # 扁平层
    flat = Flatten()(pool5)

    # 全联接层
    fc1 = Dense(4096, activation='relu')(flat)
    fc2 = Dense(4096, activation='relu')(fc1)

    # 输出层
    outputs = Dense(10, activation='softmax')(fc2)


    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy", 
              metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = x_test[1:8], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('tf2_mnist_classifier_quantized.h5')
    !vai_c_tensorflow2 \
            --model ./tf2_mnist_classifier_quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel \
            --net_name deploy
    os.rename("./xmodel/deploy.xmodel","./xmodel/mnist_classifier_vgg16_{input_x}_{input_y}.xmodel".format(input_x=input_x,input_y=input_y))

In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize



shape_range =range(32,512,32)
for shape_x in shape_range:
    shape_y = shape_x
#     X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
#     Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
#     X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
#     X_train /= 255.0
    X_test /= 255.0
    inputs = Input(shape=(shape_x, shape_y, 1))
    for step in range(1,11):
        # 卷积层和最大池化层
        conv1_kernel_num = random.randrange(32,64,2)
        conv1 = Conv2D(conv1_kernel_num, (3,3), padding='same', activation='relu')(inputs)
        conv2_kernel_num = random.randrange(conv1_kernel_num,64,2)
        conv2 = Conv2D(conv2_kernel_num, (3,3), padding='same', activation='relu')(conv1)
        pool1 = MaxPooling2D(pool_size=2)(conv2)

        conv3_kernel_num = random.randrange(conv2_kernel_num,128,2)
        conv3 = Conv2D(conv3_kernel_num, (3,3), padding='same', activation='relu')(pool1)
        conv4_kernel_num = random.randrange(conv3_kernel_num,128,2)
        conv4 = Conv2D(conv4_kernel_num, (3,3), padding='same', activation='relu')(conv3)
        pool2 = MaxPooling2D(pool_size=2)(conv4)

        conv5_kernel_num = random.randrange(conv4_kernel_num,256,2)
        conv5 = Conv2D(conv5_kernel_num, (3,3), padding='same', activation='relu')(pool2)
        conv6_kernel_num = random.randrange(conv5_kernel_num,256,2)
        conv6 = Conv2D(conv6_kernel_num, (3,3), padding='same', activation='relu')(conv5)
        conv7_kernel_num = random.randrange(conv6_kernel_num,256,2)
        conv7 = Conv2D(conv7_kernel_num, (3,3), padding='same', activation='relu')(conv6)
        pool3 = MaxPooling2D(pool_size=2)(conv7)

        conv8_kernel_num = random.randrange(conv7_kernel_num,512,2)
        conv8 = Conv2D(conv8_kernel_num, (3,3), padding='same', activation='relu')(pool3)
        conv9_kernel_num = random.randrange(conv8_kernel_num,512,2)
        conv9 = Conv2D(conv9_kernel_num, (3,3), padding='same', activation='relu')(conv8)
        conv10_kernel_num = random.randrange(conv9_kernel_num,512,2)
        conv10 = Conv2D(conv10_kernel_num, (3,3), padding='same', activation='relu')(conv9)
        pool4 = MaxPooling2D(pool_size=2)(conv10)

        conv11_kernel_num = random.randrange(conv10_kernel_num,512,2)
        conv11 = Conv2D(conv11_kernel_num, (3,3), padding='same', activation='relu')(pool4)
        conv12_kernel_num = random.randrange(conv11_kernel_num,512,2)
        conv12 = Conv2D(conv12_kernel_num, (3,3), padding='same', activation='relu')(conv11)
        conv13_kernel_num = random.randrange(128,256,2)
        conv13 = Conv2D(conv13_kernel_num, (3,3), padding='same', activation='relu')(conv12)
        pool5 = MaxPooling2D(pool_size=2)(conv13)

        # 扁平层
        flat = Flatten()(pool5)

        # 全联接层
        Dense1_neuron_num = random.randrange(1072,3072,32)
        fc1 = Dense(Dense1_neuron_num, activation='relu')(flat)
        Dense2_neuron_num = random.randrange(1072,Dense1_neuron_num+1,32)
        fc2 = Dense(Dense2_neuron_num, activation='relu')(fc1)

        # 输出层
        outputs = Dense(10, activation='softmax')(fc2)


        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        model.save('./vgg16_h5/vgg16_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))

        #model.compile(optimizer='adam',loss="sparse_categorical_crossentropy", metrics=['accuracy'])
        #history = model.fit(x_train, y_train, epochs=1)
        #model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)

        #quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg16 \
                --net_name deploy
        os.rename("./xmodel_vgg16/deploy.xmodel","./xmodel_vgg16/vgg16_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step))

2022-06-11 02:34:35.591567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-06-11 02:34:35.591599: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 50)        14450     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 52)        23452     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 60)        28140     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 60)          0     

2022-06-11 02:34:36.629068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-06-11 02:34:36.629099: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-11 02:34:36.629117: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-06-11 02:34:36.629253: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 322ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2022-06-11 02:34:46.476184: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 67/67 [00:00<00:00, 278.31it/s]             
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B2304_MAX_BG2
[UNILOG][INFO] Graph name: model, with op num: 125
[UNILO

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 280ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

KeyboardInterrupt: 